# Table of model performance 

Load data of accuracy of each model trained including baseline and generate results table

In [2]:
import os
import pandas as pd
from directory_paths import SAVE_DIR
from train.train_model_utils import *

# From saved data

In [4]:
path_to_data = os.path.join(
    SAVE_DIR, 
    "0_Zenodo",
    "data_for_figs",
    "model_accuracy_results.csv")
df_results = pd.read_csv(path_to_data)

In [5]:
# Define a function to format a float to 2 decimal places or return an empty string for NaN
def format_float(val):
    if isinstance(val, float):
        return f"{val:.2f}"
    else:
        return val

# Round floats to 2 decimal places in the DataFrame
df_results_rounded = df_results.copy()
for col in df_results.columns[3:]:  # Format numeric columns starting from 'Accuracy pcr'
    df_results_rounded[col] = df_results[col].apply(format_float)

# Create LaTeX table
latex_table = df_results_rounded.to_latex(index=False, escape=False)

# Print the LaTeX table
print(latex_table)


\begin{tabular}{lllllllll}
\toprule
Model & Inputs & Outputs & Accuracy pcr & Accuracy elbow pcr & Accuracy flag & Accuracy elbow flag & Accuracy elbow-flex & Accuracy elbow elbow-flex \\
\midrule
Sandbrink, Mamidanna, et. al. 2023 & length, velocity & wrist xyz & 0.13 & nan & nan & nan & nan & nan \\
eLife & length, velocity & wrist xyz & 0.15055 & nan & nan & nan & 1.5310975313186646 & nan \\
pcr-joints & length, velocity & wrist xyz and angles & 0.2072055618229665 & 0.1170054640817014 & 12.38701313192194 & 21.269576939669523 & 4.533994197845459 & 3.554884433746338 \\
flag-pcr-joints & length, velocity & wrist xyz and angles & 0.2248697465187625 & 0.161919738509153 & 0.5574182949282906 & 0.4578820710832422 & 1.352242946624756 & 0.5632959008216858 \\
flag-pcr-joints-spindles & 5 Ia, 5 II afferents & wrist xyz and angles & 0.60696269022791 & 0.4715672927467446 & 1.6718256907029585 & 1.3170919689265164 & 3.233213424682617 & 2.9257819652557373 \\
flag-pcr-joints-spindles & 5 Ia, 5 II aff

# Or extract data from the models

In [ ]:
# Define model paths, names, inputs, and outputs
paths_to_models = [
    "experiment_eLife_repletter_reconstruction/spatiotemporal_4_8-8-32-64_7272_0",
    "experiment_eLife_rep_pcr_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0",
    "experiment_eLife_rep_flag-pcr_l-v_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0",
    "experiment_causal_flag-pcr_optimizedLinearFR_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0",
]
model_names = ["eLife", "pcr-joints", "flag-pcr-joints", "flag-pcr-joints-spindles"]
inputs = ["length, velocity", "length, velocity", "length, velocity", "5 Ia, 5 II afferents"]
outputs = ["wrist xyz", "wrist xyz and angles", "wrist xyz and angles", "wrist xyz and angles"]

path_to_accuracy_flag_pcr = "test/FLAG_PCR/240Hz/accuracy.txt"
path_to_accuracy_elbow = "test/ELBOW-FLEX/240Hz/accuracy.txt"
path_to_accuracy_flag_pcr_66 = "test/FLAG_PCR/66_7Hz/accuracy.txt"
path_to_accuracy_elbow_66 = "test/ELBOW-FLEX/66_7Hz/accuracy.txt"
paths_to_accuracy = [path_to_accuracy_flag_pcr, path_to_accuracy_elbow, path_to_accuracy_flag_pcr_66, path_to_accuracy_elbow_66]
accuracy_labels = ["FLAG-PCR", "ELBOW-FLEX", "FLAG-PCR", "ELBOW-FLEX"]


## Default models with single seed

In [37]:

# Initialize results DataFrame
df_results = pd.DataFrame(columns=[
    "Model", "Inputs", "Outputs", 
    "Accuracy pcr", "Accuracy elbow pcr", 
    "Accuracy flag", "Accuracy elbow flag", 
    "Accuracy elbow-flex", "Accuracy elbow elbow-flex"
])

# add results from the Sandbrink paper
df_results.loc[len(df_results)] = [
    "Sandbrink, Mamidanna, et. al. 2023", inputs[0], outputs[0],
    0.13, None, None, None, None, None
]

# Iterate over models
for model_idx, path in enumerate(paths_to_models):
    model_path = os.path.join(SAVE_DIR, "trained_models", path)
    # Initialize metrics for this model
    acc_pcr = acc_elbow_pcr = acc_flag = acc_elbow_flag = None
    acc_elbow_flex = acc_elbow_elbow_flex = None
    
    # Iterate over accuracy paths
    for i, path_to_accuracy in enumerate(paths_to_accuracy):
        accuracy_path = os.path.join(model_path, path_to_accuracy)
        print(f"Processing file: {accuracy_path}")
        
        # Check if the file exists
        if not os.path.exists(accuracy_path):
            print(f"File not found: {accuracy_path}")
            continue

        try:
            # Read the file line by line into a list of tuples
            data = []
            with open(accuracy_path, 'r') as f:
                for line in f:
                    if ":" in line:  # Only process lines with the delimiter
                        key, value = line.split(":", 1)
                        key = key.strip()
                        value = value.strip()
                        data.append((key, value if value != "None" else None))  # Replace "None" with None

            # Create a DataFrame from the parsed data
            df = pd.DataFrame(data, columns=["Metric", "Value"])
            df["Value"] = pd.to_numeric(df["Value"], errors="coerce")  # Convert numeric values; leave others as NaN

            print(df)  # Debug output to inspect the DataFrame
        except Exception as e:
            print(f"Error processing file {accuracy_path}: {e}")
            continue

        # Extract metrics based on accuracy label
        if accuracy_labels[i] == "FLAG-PCR":
            acc_flag = df.loc[df["Metric"] == "split_accuracy_first_n", "Value"].squeeze()
            acc_pcr = df.loc[df["Metric"] == "split_accuracy_rest", "Value"].squeeze()
            acc_elbow_flag = df.loc[df["Metric"] == "split_elbow_first_n", "Value"].squeeze()
            acc_elbow_pcr = df.loc[df["Metric"] == "split_elbow_rest", "Value"].squeeze()
        elif accuracy_labels[i] == "ELBOW-FLEX":
            acc_elbow_flex = df.loc[df["Metric"] == "overall_accuracy", "Value"].squeeze()
            acc_elbow_elbow_flex = df.loc[df["Metric"] == "elbow_angle_loss", "Value"].squeeze()

    # Append to results DataFrame
    df_results.loc[len(df_results)] = [
        model_names[model_idx], inputs[model_idx], outputs[model_idx],
        acc_pcr, acc_elbow_pcr, acc_flag, acc_elbow_flag,
        acc_elbow_flex, acc_elbow_elbow_flex
    ]
# add extra rows



# Display results
# print(df_results)

Processing file: /media/data16/adriana/ProprioPerception/trained_models/experiment_eLife_repletter_reconstruction/spatiotemporal_4_8-8-32-64_7272_0/test/FLAG_PCR/240Hz/accuracy.txt
File not found: /media/data16/adriana/ProprioPerception/trained_models/experiment_eLife_repletter_reconstruction/spatiotemporal_4_8-8-32-64_7272_0/test/FLAG_PCR/240Hz/accuracy.txt
Processing file: /media/data16/adriana/ProprioPerception/trained_models/experiment_eLife_repletter_reconstruction/spatiotemporal_4_8-8-32-64_7272_0/test/ELBOW-FLEX/240Hz/accuracy.txt
File not found: /media/data16/adriana/ProprioPerception/trained_models/experiment_eLife_repletter_reconstruction/spatiotemporal_4_8-8-32-64_7272_0/test/ELBOW-FLEX/240Hz/accuracy.txt
Processing file: /media/data16/adriana/ProprioPerception/trained_models/experiment_eLife_repletter_reconstruction/spatiotemporal_4_8-8-32-64_7272_0/test/FLAG_PCR/66_7Hz/accuracy.txt
File not found: /media/data16/adriana/ProprioPerception/trained_models/experiment_eLife_repl

In [38]:
df_results

,Model,Inputs,Outputs,Accuracy pcr,Accuracy elbow pcr,Accuracy flag,Accuracy elbow flag,Accuracy elbow-flex,Accuracy elbow elbow-flex
0,"Sandbrink, Mamidanna, et. al. 2023","length, velocity",wrist xyz,0.130000,NaN,NaN,NaN,NaN,NaN
1,eLife,"length, velocity",wrist xyz,NaN,NaN,NaN,NaN,1.531098,NaN
2,pcr-joints,"length, velocity",wrist xyz and angles,0.207206,0.117005,12.387013,21.269577,4.533994,3.554884
3,flag-pcr-joints,"length, velocity",wrist xyz and angles,0.224870,0.161920,0.557418,0.457882,1.352243,0.563296
4,flag-pcr-joints-spindles,"5 Ia, 5 II afferents",wrist xyz and angles,0.606963,0.471567,1.671826,1.317092,3.233213,2.925782


In [39]:
# from previous calcluation
row_copy = df_results.iloc[1].copy()
row_copy["Accuracy pcr"] = 0.15055
df_results.iloc[1] = row_copy

## Add variance across params

In [40]:
from scipy import stats

path_to_models = "experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints"

model_names = ["flag-pcr-joints-spindles"]
inputs = ["5 Ia, 5 II afferents"]
outputs = ["wrist xyz and angles"]

models_seed_to_use = ["0", "1", "2", "3", "4"]
train_seeds_to_use = ["9"]

# Initialize lists to collect accuracy values
acc_pcr_list = []
acc_elbow_pcr_list = []
acc_flag_list = []
acc_elbow_flag_list = []
acc_elbow_flex_list = []
acc_elbow_elbow_flex_list = []

# Iterate over directories found in paths_to_models
base_path = os.path.join(SAVE_DIR, "trained_models", path_to_models)
# Iterate over all directories in the given path
for dir_name in os.listdir(base_path):
    dir_path = os.path.join(base_path, dir_name)
    if os.path.isdir(dir_path):  # Check if it's a directory
        print(dir_path)  # Do something with the directory path
        model_path = dir_path
        
        model_seed = model_path.split("_")[-2]  # Extract the model seed from the directory name
        train_seed = model_path.split("_")[-1]  # Extract the training seed from the directory name
        # Check if the model seed and training seed are in the specified lists
        if model_seed not in models_seed_to_use or train_seed not in train_seeds_to_use:
            continue
        # Iterate over accuracy paths
        for i, path_to_accuracy in enumerate(paths_to_accuracy):
            accuracy_path = os.path.join(model_path, path_to_accuracy)
            print(f"Processing file: {accuracy_path}")
            
            # Check if the file exists
            if not os.path.exists(accuracy_path):
                print(f"File not found: {accuracy_path}")
                continue

            try:
                # Read the file line by line into a list of tuples
                data = []
                with open(accuracy_path, 'r') as f:
                    for line in f:
                        if ":" in line:  # Only process lines with the delimiter
                            key, value = line.split(":", 1)
                            key = key.strip()
                            value = value.strip()
                            data.append((key, value if value != "None" else None))  # Replace "None" with None

                # Create a DataFrame from the parsed data
                df = pd.DataFrame(data, columns=["Metric", "Value"])
                df["Value"] = pd.to_numeric(df["Value"], errors="coerce")  # Convert numeric values; leave others as NaN

                print(df)  # Debug output to inspect the DataFrame
            except Exception as e:
                print(f"Error processing file {accuracy_path}: {e}")
                continue

            # Extract metrics based on accuracy label
            if accuracy_labels[i] == "FLAG-PCR":
                acc_flag = df.loc[df["Metric"] == "split_accuracy_first_n", "Value"].squeeze()
                acc_pcr = df.loc[df["Metric"] == "split_accuracy_rest", "Value"].squeeze()
                acc_elbow_flag = df.loc[df["Metric"] == "split_elbow_first_n", "Value"].squeeze()
                acc_elbow_pcr = df.loc[df["Metric"] == "split_elbow_rest", "Value"].squeeze()
                # Add values to the respective lists
                acc_flag_list.append(acc_flag)
                acc_pcr_list.append(acc_pcr)
                acc_elbow_flag_list.append(acc_elbow_flag)
                acc_elbow_pcr_list.append(acc_elbow_pcr)
            elif accuracy_labels[i] == "ELBOW-FLEX":
                acc_elbow_flex = df.loc[df["Metric"] == "overall_accuracy", "Value"].squeeze()
                acc_elbow_elbow_flex = df.loc[df["Metric"] == "elbow_angle_loss", "Value"].squeeze()
                # Add values to the respective lists
                acc_elbow_flex_list.append(acc_elbow_flex)
                acc_elbow_elbow_flex_list.append(acc_elbow_elbow_flex)

print(acc_pcr_list)
print()
# Compute mean and SEM for each metric
acc_pcr_mean = np.mean(acc_pcr_list) if acc_pcr_list else None
acc_pcr_sem = stats.sem(acc_pcr_list, nan_policy='omit') if acc_pcr_list else None
acc_elbow_pcr_mean = np.mean(acc_elbow_pcr_list) if acc_elbow_pcr_list else None
acc_elbow_pcr_sem = stats.sem(acc_elbow_pcr_list, nan_policy='omit') if acc_elbow_pcr_list else None
acc_flag_mean = np.mean(acc_flag_list) if acc_flag_list else None
acc_flag_sem = stats.sem(acc_flag_list, nan_policy='omit') if acc_flag_list else None
acc_elbow_flag_mean = np.mean(acc_elbow_flag_list) if acc_elbow_flag_list else None
acc_elbow_flag_sem = stats.sem(acc_elbow_flag_list, nan_policy='omit') if acc_elbow_flag_list else None
acc_elbow_flex_mean = np.mean(acc_elbow_flex_list) if acc_elbow_flex_list else None
acc_elbow_flex_sem = stats.sem(acc_elbow_flex_list, nan_policy='omit') if acc_elbow_flex_list else None
acc_elbow_elbow_flex_mean = np.mean(acc_elbow_elbow_flex_list) if acc_elbow_elbow_flex_list else None
acc_elbow_elbow_flex_sem = stats.sem(acc_elbow_elbow_flex_list, nan_policy='omit') if acc_elbow_elbow_flex_list else None

print(acc_elbow_elbow_flex_mean, acc_elbow_elbow_flex_sem)
# Append computed statistics to results DataFrame
df_results.loc[len(df_results)] = [
    model_names[0], inputs[0], outputs[0],
    f"{acc_pcr_mean:.2f} ± {acc_pcr_sem:.2f}" if acc_pcr_mean is not None else None,
    f"{acc_elbow_pcr_mean:.2f} ± {acc_elbow_pcr_sem:.2f}" if acc_elbow_pcr_mean is not None else None,
    f"{acc_flag_mean:.2f} ± {acc_flag_sem:.2f}" if acc_flag_mean is not None else None,
    f"{acc_elbow_flag_mean:.2f} ± {acc_elbow_flag_sem:.2f}" if acc_elbow_flag_mean is not None else None,
    f"{acc_elbow_flex_mean:.2f} ± {acc_elbow_flex_sem:.2f}" if acc_elbow_flex_mean is not None else None,
    f"{acc_elbow_elbow_flex_mean:.2f} ± {acc_elbow_elbow_flex_sem:.2f}" if acc_elbow_elbow_flex_mean is not None else None,
]

# Optionally, save the results to a CSV file

# print(df_results)

/media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0_1
/media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_3_0
/media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/plots
/media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0_3
/media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stri

In [41]:
df_results

,Model,Inputs,Outputs,Accuracy pcr,Accuracy elbow pcr,Accuracy flag,Accuracy elbow flag,Accuracy elbow-flex,Accuracy elbow elbow-flex
0,"Sandbrink, Mamidanna, et. al. 2023","length, velocity",wrist xyz,0.13,NaN,NaN,NaN,NaN,NaN
1,eLife,"length, velocity",wrist xyz,0.15055,NaN,NaN,NaN,1.531098,NaN
2,pcr-joints,"length, velocity",wrist xyz and angles,0.207206,0.117005,12.387013,21.269577,4.533994,3.554884
3,flag-pcr-joints,"length, velocity",wrist xyz and angles,0.22487,0.16192,0.557418,0.457882,1.352243,0.563296
4,flag-pcr-joints-spindles,"5 Ia, 5 II afferents",wrist xyz and angles,0.606963,0.471567,1.671826,1.317092,3.233213,2.925782
5,flag-pcr-joints-spindles,"5 Ia, 5 II afferents",wrist xyz and angles,0.53 ± 0.01,0.52 ± 0.02,1.58 ± 0.03,1.38 ± 0.04,3.07 ± 0.11,2.87 ± 0.16


## Variance across parameters and train seeds

In [42]:
from scipy import stats

path_to_models = "experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints"

model_names = ["flag-pcr-joints-spindles"]
inputs = ["5 Ia, 5 II afferents"]
outputs = ["wrist xyz and angles"]

models_seed_to_use = ["0", "1", "2", "3", "4"]
train_seeds_to_use = ["0", "1", "2", "9"]

# Initialize lists to collect accuracy values
acc_pcr_list = []
acc_elbow_pcr_list = []
acc_flag_list = []
acc_elbow_flag_list = []
acc_elbow_flex_list = []
acc_elbow_elbow_flex_list = []

# Iterate over directories found in paths_to_models
base_path = os.path.join(SAVE_DIR, "trained_models", path_to_models)
# Iterate over all directories in the given path
for dir_name in os.listdir(base_path):
    dir_path = os.path.join(base_path, dir_name)
    if os.path.isdir(dir_path):  # Check if it's a directory
        print(dir_path)  # Do something with the directory path
        model_path = dir_path
        
        model_seed = model_path.split("_")[-2]  # Extract the model seed from the directory name
        train_seed = model_path.split("_")[-1]  # Extract the training seed from the directory name
        # Check if the model seed and training seed are in the specified lists
        if model_seed not in models_seed_to_use or train_seed not in train_seeds_to_use:
            continue
        # Iterate over accuracy paths
        for i, path_to_accuracy in enumerate(paths_to_accuracy):
            accuracy_path = os.path.join(model_path, path_to_accuracy)
            print(f"Processing file: {accuracy_path}")
            
            # Check if the file exists
            if not os.path.exists(accuracy_path):
                print(f"File not found: {accuracy_path}")
                continue

            try:
                # Read the file line by line into a list of tuples
                data = []
                with open(accuracy_path, 'r') as f:
                    for line in f:
                        if ":" in line:  # Only process lines with the delimiter
                            key, value = line.split(":", 1)
                            key = key.strip()
                            value = value.strip()
                            data.append((key, value if value != "None" else None))  # Replace "None" with None

                # Create a DataFrame from the parsed data
                df = pd.DataFrame(data, columns=["Metric", "Value"])
                df["Value"] = pd.to_numeric(df["Value"], errors="coerce")  # Convert numeric values; leave others as NaN

                print(df)  # Debug output to inspect the DataFrame
            except Exception as e:
                print(f"Error processing file {accuracy_path}: {e}")
                continue

            # Extract metrics based on accuracy label
            if accuracy_labels[i] == "FLAG-PCR":
                acc_flag = df.loc[df["Metric"] == "split_accuracy_first_n", "Value"].squeeze()
                acc_pcr = df.loc[df["Metric"] == "split_accuracy_rest", "Value"].squeeze()
                acc_elbow_flag = df.loc[df["Metric"] == "split_elbow_first_n", "Value"].squeeze()
                acc_elbow_pcr = df.loc[df["Metric"] == "split_elbow_rest", "Value"].squeeze()
                # Add values to the respective lists
                acc_flag_list.append(acc_flag)
                acc_pcr_list.append(acc_pcr)
                acc_elbow_flag_list.append(acc_elbow_flag)
                acc_elbow_pcr_list.append(acc_elbow_pcr)
            elif accuracy_labels[i] == "ELBOW-FLEX":
                acc_elbow_flex = df.loc[df["Metric"] == "overall_accuracy", "Value"].squeeze()
                acc_elbow_elbow_flex = df.loc[df["Metric"] == "elbow_angle_loss", "Value"].squeeze()
                # Add values to the respective lists
                acc_elbow_flex_list.append(acc_elbow_flex)
                acc_elbow_elbow_flex_list.append(acc_elbow_elbow_flex)

print(acc_pcr_list)
print()
# Compute mean and SEM for each metric
acc_pcr_mean = np.mean(acc_pcr_list) if acc_pcr_list else None
acc_pcr_sem = stats.sem(acc_pcr_list, nan_policy='omit') if acc_pcr_list else None
acc_elbow_pcr_mean = np.mean(acc_elbow_pcr_list) if acc_elbow_pcr_list else None
acc_elbow_pcr_sem = stats.sem(acc_elbow_pcr_list, nan_policy='omit') if acc_elbow_pcr_list else None
acc_flag_mean = np.mean(acc_flag_list) if acc_flag_list else None
acc_flag_sem = stats.sem(acc_flag_list, nan_policy='omit') if acc_flag_list else None
acc_elbow_flag_mean = np.mean(acc_elbow_flag_list) if acc_elbow_flag_list else None
acc_elbow_flag_sem = stats.sem(acc_elbow_flag_list, nan_policy='omit') if acc_elbow_flag_list else None
acc_elbow_flex_mean = np.mean(acc_elbow_flex_list) if acc_elbow_flex_list else None
acc_elbow_flex_sem = stats.sem(acc_elbow_flex_list, nan_policy='omit') if acc_elbow_flex_list else None
acc_elbow_elbow_flex_mean = np.mean(acc_elbow_elbow_flex_list) if acc_elbow_elbow_flex_list else None
acc_elbow_elbow_flex_sem = stats.sem(acc_elbow_elbow_flex_list, nan_policy='omit') if acc_elbow_elbow_flex_list else None

print(acc_elbow_elbow_flex_mean, acc_elbow_elbow_flex_sem)
# Append computed statistics to results DataFrame
df_results.loc[len(df_results)] = [
    model_names[0], inputs[0], outputs[0],
    f"{acc_pcr_mean:.2f} ± {acc_pcr_sem:.2f}" if acc_pcr_mean is not None else None,
    f"{acc_elbow_pcr_mean:.2f} ± {acc_elbow_pcr_sem:.2f}" if acc_elbow_pcr_mean is not None else None,
    f"{acc_flag_mean:.2f} ± {acc_flag_sem:.2f}" if acc_flag_mean is not None else None,
    f"{acc_elbow_flag_mean:.2f} ± {acc_elbow_flag_sem:.2f}" if acc_elbow_flag_mean is not None else None,
    f"{acc_elbow_flex_mean:.2f} ± {acc_elbow_flex_sem:.2f}" if acc_elbow_flex_mean is not None else None,
    f"{acc_elbow_elbow_flex_mean:.2f} ± {acc_elbow_elbow_flex_sem:.2f}" if acc_elbow_elbow_flex_mean is not None else None,
]

# Optionally, save the results to a CSV file
# df_results.to_csv(os.path.join(SAVE_DIR,"model_accuracy_results.csv"), index=False)
# print(df_results)

/media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0_1
Processing file: /media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0_1/test/FLAG_PCR/240Hz/accuracy.txt
                   Metric     Value
0        overall_accuracy  0.759055
1  split_accuracy_first_n  1.391233
2     split_accuracy_rest  0.576055
3     split_elbow_first_n  1.375805
4        split_elbow_rest  0.499451
5        elbow_angle_loss  0.696183
6          wrist_xyz_loss  0.759055
Processing file: /media/data16/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8

In [43]:
df_results

,Model,Inputs,Outputs,Accuracy pcr,Accuracy elbow pcr,Accuracy flag,Accuracy elbow flag,Accuracy elbow-flex,Accuracy elbow elbow-flex
0,"Sandbrink, Mamidanna, et. al. 2023","length, velocity",wrist xyz,0.13,NaN,NaN,NaN,NaN,NaN
1,eLife,"length, velocity",wrist xyz,0.15055,NaN,NaN,NaN,1.531098,NaN
2,pcr-joints,"length, velocity",wrist xyz and angles,0.207206,0.117005,12.387013,21.269577,4.533994,3.554884
3,flag-pcr-joints,"length, velocity",wrist xyz and angles,0.22487,0.16192,0.557418,0.457882,1.352243,0.563296
4,flag-pcr-joints-spindles,"5 Ia, 5 II afferents",wrist xyz and angles,0.606963,0.471567,1.671826,1.317092,3.233213,2.925782
5,flag-pcr-joints-spindles,"5 Ia, 5 II afferents",wrist xyz and angles,0.53 ± 0.01,0.52 ± 0.02,1.58 ± 0.03,1.38 ± 0.04,3.07 ± 0.11,2.87 ± 0.16
6,flag-pcr-joints-spindles,"5 Ia, 5 II afferents",wrist xyz and angles,0.57 ± 0.01,0.56 ± 0.01,1.54 ± 0.02,1.42 ± 0.03,3.06 ± 0.04,2.84 ± 0.09


In [45]:
df_results.to_csv(os.path.join(SAVE_DIR,"model_accuracy_results.csv"), index=False)

In [46]:
# Define a function to format a float to 2 decimal places or return an empty string for NaN
def format_float(val):
    if isinstance(val, float):
        return f"{val:.2f}"
    else:
        return val

# Round floats to 2 decimal places in the DataFrame
df_results_rounded = df_results.copy()
for col in df_results.columns[3:]:  # Format numeric columns starting from 'Accuracy pcr'
    df_results_rounded[col] = df_results[col].apply(format_float)

# Create LaTeX table
latex_table = df_results_rounded.to_latex(index=False, escape=False)

# Print the LaTeX table
print(latex_table)


\begin{tabular}{lllllllll}
\toprule
Model & Inputs & Outputs & Accuracy pcr & Accuracy elbow pcr & Accuracy flag & Accuracy elbow flag & Accuracy elbow-flex & Accuracy elbow elbow-flex \\
\midrule
Sandbrink, Mamidanna, et. al. 2023 & length, velocity & wrist xyz & 0.13 & nan & nan & nan & nan & nan \\
eLife & length, velocity & wrist xyz & 0.15 & nan & nan & nan & 1.53 & nan \\
pcr-joints & length, velocity & wrist xyz and angles & 0.21 & 0.12 & 12.39 & 21.27 & 4.53 & 3.55 \\
flag-pcr-joints & length, velocity & wrist xyz and angles & 0.22 & 0.16 & 0.56 & 0.46 & 1.35 & 0.56 \\
flag-pcr-joints-spindles & 5 Ia, 5 II afferents & wrist xyz and angles & 0.61 & 0.47 & 1.67 & 1.32 & 3.23 & 2.93 \\
flag-pcr-joints-spindles & 5 Ia, 5 II afferents & wrist xyz and angles & 0.53 ± 0.01 & 0.52 ± 0.02 & 1.58 ± 0.03 & 1.38 ± 0.04 & 3.07 ± 0.11 & 2.87 ± 0.16 \\
flag-pcr-joints-spindles & 5 Ia, 5 II afferents & wrist xyz and angles & 0.57 ± 0.01 & 0.56 ± 0.01 & 1.54 ± 0.02 & 1.42 ± 0.03 & 3.06 ± 0.04 &